# Flood extent using Sentinel 1

Flood extent can be determined using a change detection approach on Sentinel-1 data. In this notebook we have tried in adopting (UN SPIDER's)[https://www.un-spider.org/advisory-support/recommended-practices/recommended-practice-google-earth-engine-flood-mapping] recommendation practice for computing flood extent by implementing openeo process.

In [1]:
# import necessary packages
import openeo
from openeo.api.process import Parameter
import json
from pathlib import Path
import matplotlib.pyplot as plt
import rasterio
import numpy as np

# connect with the backend
eoconn = openeo.connect("openeo.vito.be").authenticate_oidc()

Authenticated using refresh token.


User can choose among different backend available [here](https://hub.openeo.org/) to connect to the backend. Rrgarding the authentication process OpenID connect (oidc) is recommended, but not always straightforward to use. In cases where you are unable to connect with the backend use basic authentication method explained [here](https://openeo.org/documentation/1.0/authentication.html#openid-connect).

In [2]:
# function to load geojson file
def read_json(path: Path) -> dict:
    with open(path) as input:
        field = json.load(input)
        input.close()
    return field

To use the data collection, a user must use the correct backend with the data collection. Then using load_collection, they can specify bands, temporal extent (i.e. interested time interval) and even spatial extent. In this example, we have loaded Sentinel1 data from Terrascope available in VITO backend over a specified spatial extent.

In [3]:
# load before flash flood params
before_date = ["2021-05-12","2021-05-12"]
after_date = ["2021-06-18", "2021-06-18"]
spatial_param = read_json("../aoi/cologne_all.geojson")


In [4]:
# using S1 data from Sentinelhub (https://hub.openeo.org/) directly instead of downloading

before_cube = eoconn.load_collection(
                            "SENTINEL1_GAMMA0_SENTINELHUB",
                            temporal_extent = before_date,
                            spatial_extent = spatial_param,
                            bands = ['VV'],
                            properties={"sat:orbit_state": lambda v: v=="ascending"}
                            )
after_cube = eoconn.load_collection(
                            "SENTINEL1_GAMMA0_SENTINELHUB",
                            temporal_extent = after_date,
                            spatial_extent = spatial_param,
                            bands = ['VV'],
                            properties={"sat:orbit_state": lambda v: v=="ascending"}
                            )

Since now we have details on temporal dimension we can perform dimension reduction. As we loaded our collection for specific time intervals, it can include multiple time dimensions. Thus [reduce_dimension](https://processes.openeo.org/#reduce_dimension) applies a reducer to a data cube dimension by collapsing all the pixel values along the time dimension into an output value computed by the reducer.

In [5]:
#'reduce_dimension' to reduce temporal dimension
rbefore_cube = before_cube.reduce_dimension(dimension="t", reducer="mean")
rafter_cube = after_cube.reduce_dimension(dimension="t", reducer="mean")

In [6]:
# calculating the ratio of post and pre datacube as mentioned in the UNSPIDER documentation for flood extent using Sentinel 1
difference = rafter_cube.divide(rbefore_cube)

Here we tried in presenting a method to create and use UDF as an openEO feature. In a similar manner user can create their own UDF as needed to apply to their data cube. More information on [UDF](https://open-eo.github.io/openeo-python-client/udf.html). 

Our UDF is designed to perform thresholding to the final result obtained by comparing pre and post datacubes and returns a cube that assigns 1 to region with higher value than threshold otherwise 0.

In [7]:
# define a udf that will perform thresholding of the dataset
udf = openeo.UDF("""
from openeo.udf import XarrayDataCube

def apply_datacube(cube: XarrayDataCube, context: dict) -> XarrayDataCube:
    array = cube.get_array()
    
    # UN defined difference threshold
    array.values = np.where(array > 1.5, 1, 0)
    return cube
""")

# Apply the UDF to a cube.
threshold_cube = difference.apply(process=udf)

Once the process is completed, you can also save it as your process using [save_user_defined_process](https://open-eo.github.io/openeo-python-client/udp.html) that can later be used for a similar task. Otherwise, you can download the result either by direct download (in case of the small spatial extent with few processing) or perform create a [batch job](https://open-eo.github.io/openeo-python-client/batch_jobs.html) in case it is a heavy task over a large extent.

In [8]:
# download your result either syncronous or proceed as batch job
threshold_cube.download("s1_diff.tiff")